In [1]:
import pandas as pd
import pymysql
import time
import os

In [2]:
path='data/'

### diary2

In [3]:
d1 = [i for i in os.listdir(path) if i.startswith('diary_3')]
len(d1)

26

In [4]:
kp=[]
for i in d1:
    sz=os.stat(path+i).st_size/1024/1024.0
    print i.rjust(17),sz
    if sz>1.5:
        kp.append(i)
len(kp)

   diary_3_15.csv 2.697265625
   diary_3_18.csv 2.7060546875
   diary_3_13.csv 2.693359375
   diary_3_16.csv 2.6982421875
   diary_3_10.csv 2.681640625
   diary_3_25.csv 3.14453125
   diary_3_22.csv 2.60546875
   diary_3_32.csv 4.2314453125
   diary_3_28.csv 4.283203125
   diary_3_34.csv 4.2548828125
    diary_3_1.csv 4.50390625
   diary_3_14.csv 2.693359375
   diary_3_27.csv 1.3583984375
   diary_3_17.csv 2.7060546875
    diary_3_7.csv 4.615234375
   diary_3_26.csv 3.0107421875
   diary_3_20.csv 2.7041015625
   diary_3_31.csv 4.2880859375
   diary_3_19.csv 2.7060546875
   diary_3_35.csv 0.962890625
   diary_3_23.csv 3.1416015625
   diary_3_33.csv 4.255859375
   diary_3_11.csv 2.6865234375
    diary_3_9.csv 2.6796875
   diary_3_12.csv 2.69140625
    diary_3_5.csv 4.5380859375


24

In [5]:
%%time
coll=[]
for i in kp:
    df0 = pd.read_csv(path+i)
    df0['serial']=int(i.split('_')[-1].split('.csv')[0])
    coll.append(df0)
    del df0
print len(coll)

24
CPU times: user 1.14 s, sys: 136 ms, total: 1.28 s
Wall time: 1.43 s


In [6]:
df = pd.concat(coll,axis=0)
df.shape

(4372467, 5)

### post_ID is not null

In [7]:
df = df[(~df['post_ID'].isnull())]
df.shape

(881538, 5)

In [8]:
df['serial'].value_counts()

33    176598
34    176549
31    176514
28    176272
32    175605
Name: serial, dtype: int64

In [9]:
seq = sorted(df['serial'].unique())
seq

[28, 31, 32, 33, 34]

In [10]:
date = ['2019-02-21','2018-02-22','2018-02-23','2018-02-24','2018-02-25']
date

['2019-02-21', '2018-02-22', '2018-02-23', '2018-02-24', '2018-02-25']

In [11]:
times = {i:j for i,j in zip(seq,date)}
times

{28: '2019-02-21',
 31: '2018-02-22',
 32: '2018-02-23',
 33: '2018-02-24',
 34: '2018-02-25'}

In [12]:
df['date']=df['serial'].map(times)

In [13]:
df['post_ID'] = df['post_ID'].astype('int')

In [14]:
df.dtypes

post_ID      int64
comments     int64
favor        int64
views        int64
serial       int64
date        object
dtype: object

In [15]:
df.shape

(881538, 6)

In [16]:
df = df.drop_duplicates()
df.shape

(881536, 6)

In [17]:
df2 = df[['post_ID','date','comments','favor','views']]
df2.shape

(881536, 5)

In [18]:
df['post_ID'].value_counts().head(3)

14682111    5
13478766    5
14302108    5
Name: post_ID, dtype: int64

In [19]:
df2.head()

,post_ID,date,comments,favor,views
0,8145114,2018-02-23,11,2,2924
1,8352256,2018-02-23,0,0,574
2,8145298,2018-02-23,1,1,907
3,8290294,2018-02-23,23,11,5023
4,8196685,2018-02-23,1,2,3352


In [20]:
from secret import dburl, dbusername, dbpasswd

In [21]:
db = pymysql.connect(host=dburl,    # your host, usually localhost
                     user=dbusername,         # your username
                     passwd=dbpasswd,  # your password
                     port=3306,
                     db="research",
                     charset='utf8')

In [22]:
cur=db.cursor()

In [23]:
from time import time

In [24]:
def BatchInsert(df):
    df=df[['post_ID','date','comments','favor','views']]
    batch = [df.iloc[n*10000:(n+1)*10000,:] for n in range(0 ,int(df.shape[0]/10000.0)+1)]
    
    start=time()
    n=0
    for d in batch:
        
        sql = """insert ignore into monitorpost values {}""".format(str(map(lambda x: tuple(x),d.values))[1:-1])
        n+=1
        cur.execute(sql)
        db.commit()
        print n,'{:.2f}s |'.format(time()-start),
        start=time()

    

In [25]:
df.shape

(881536, 6)

In [26]:
%%time
BatchInsert(df2)

1 1.85s | 2 1.80s | 3 1.76s | 4 1.78s | 5 1.81s | 6 1.83s | 7 1.79s | 8 1.96s | 9 1.82s | 10 1.81s | 11 1.84s | 12 1.78s | 13 1.82s | 14 1.75s | 15 1.79s | 16 1.77s | 17 1.74s | 18 1.78s | 19 1.83s | 20 1.81s | 21 1.74s | 22 1.76s | 23 1.77s | 24 1.80s | 25 1.77s | 26 1.78s | 27 1.76s | 28 1.77s | 29 1.79s | 30 1.89s | 31 1.85s | 32 1.79s | 33 1.77s | 34 1.76s | 35 1.73s | 36 1.76s | 37 1.81s | 38 1.81s | 39 1.78s | 40 1.82s | 41 1.88s | 42 1.76s | 43 1.82s | 44 1.88s | 45 1.81s | 46 1.86s | 47 1.88s | 48 1.79s | 49 1.78s | 50 1.76s | 51 1.85s | 52 1.88s | 53 1.95s | 54 1.90s | 55 1.83s | 56 1.98s | 57 2.43s | 58 2.07s | 59 1.82s | 60 1.88s | 61 1.93s | 62 1.97s | 63 1.81s | 64 1.87s | 65 1.82s | 66 1.90s | 67 1.82s | 68 1.81s | 69 1.85s | 70 1.79s | 71 1.78s | 72 1.83s | 73 1.78s | 74 1.84s | 75 1.89s | 76 1.82s | 77 1.82s | 78 1.80s | 79 1.77s | 80 1.79s | 81 1.76s | 82 1.76s | 83 1.73s | 84 1.73s | 85 1.80s | 86 1.76s | 87 1.73s | 88 1.74s | 89 0.31s |CPU times: user 2.52 s, sys: 12

In [51]:
def SingleInsert(df):
    df=df[['post_ID','date','comments','favor','views']]
    
    start=time()
    n=0
    for d in df.values:
        
        sql = """insert into monitorpost values {}""".format(str(tuple(d)))
        n+=1
        try:
            cur.execute(sql)
        except Exception as e:
            print e
        if n%10000==0:
            print n,'{:.2f}s |'.format(time()-start),
            start=time()

    db.commit()
    print 'commit~ done!'


In [52]:
%%time
SingleInsert(df)

(1062, u"Duplicate entry '8145114-2018-02-23' for key 'PRIMARY'")
(1062, u"Duplicate entry '8352256-2018-02-23' for key 'PRIMARY'")
(1062, u"Duplicate entry '8145298-2018-02-23' for key 'PRIMARY'")
(1062, u"Duplicate entry '8290294-2018-02-23' for key 'PRIMARY'")
(1062, u"Duplicate entry '8196685-2018-02-23' for key 'PRIMARY'")
(1062, u"Duplicate entry '8167356-2018-02-23' for key 'PRIMARY'")
(1062, u"Duplicate entry '8153949-2018-02-23' for key 'PRIMARY'")
10000 302.04s | 20000 389.48s | 30000 440.99s | 40000 301.02s |

KeyboardInterrupt: 